In [0]:
# create a dropdown widget from where I will chose excel or csv
dbutils.widgets.dropdown("source_extention", "excel", ["excel", "csv"]) 
#get that widget value and save in a variable name
source_extention = dbutils.widgets.get("source_extention")

In [0]:
%sql
--drop table dev_edh.dummy.delta_source

In [0]:
%sql
use catalog dev_edh;
use schema dummy;
--select count(*) from dev_edh.dummy.delta_source;
--truncate TABLE dev_poc.default.delta_source;



In [0]:
import pandas as pd
import warnings
import time
warnings.simplefilter(action='ignore', category=FutureWarning)
dbutils.widgets.text(label="Source Directory", name="source_path", defaultValue='')


The code iterates through files in a specified directory, filtering files based on the extension stored in the variable 'source_extention'.
For each matching file, it reads all sheets in the Excel file.
For each sheet, it reads the data into a DataFrame, retrieves the file's modification time, and records the current load time.
It converts the sheet data to JSON format and appends a list containing the file path, sheet name, modification time, load time, and data to a list.
Then, it creates a Spark DataFrame from this list with specified columns.


In [0]:
import os
import pandas as pd
import time
from pyspark.sql.types import StructType, StructField, StringType

data = []  # Initialize an empty list to store data
cols = ['FileName', 'SheetName', 'ModificationTime', 'LoadTime', 'Data']  # Define column names for the DataFrame
directory = dbutils.widgets.get("source_path")  # Get the directory path from the widget
list_files = os.listdir(directory)  # List all files in the directory

for item in list_files:
    file_path = os.path.join(directory, item)  # Get the full file path
    # Check if the file extension matches the selected source extension
    if source_extention == "excel" and item.endswith('.xlsx'):
        sheets = pd.ExcelFile(file_path)  # Read the Excel file
        if len(sheets.sheet_names) >= 1:  # Check if there is at least one sheet
            for sheet in sheets.sheet_names:  # Iterate through each sheet
                pdf = pd.read_excel(file_path, sheet_name=sheet)  # Read the sheet into a DataFrame
                modfied_time = int(os.path.getmtime(file_path) / 1000)  # Get the file modification time
                value = pdf.to_json(orient='split')  # Convert the DataFrame to JSON
                load_time = time.strftime("%Y%m%d:%H:%M:%S", time.localtime())  # Get the current load time

                # Append the data to the list
                data.append([file_path, sheet, time.strftime('%Y%m%d:%H:%M:%S', time.localtime(modfied_time)), load_time, value])
            df = spark.createDataFrame(data, cols)  # Create a Spark DataFrame from the data list    
    elif source_extention == "csv" and item.endswith('.csv'):
        pdf = pd.read_csv(file_path, encoding='ISO-8859-1')  # Read the CSV file into a DataFrame
        modfied_time = int(os.path.getmtime(file_path) / 1000)  # Get the file modification time
        value = pdf.to_json(orient='split')  # Convert the DataFrame to JSON
        load_time = time.strftime("%Y%m%d:%H:%M:%S", time.localtime())  # Get the current load time

        # Append the data to the list
        data.append([file_path, None, time.strftime('%Y%m%d:%H:%M:%S', time.localtime(modfied_time)), load_time, value])

schema = StructType([
    StructField('FileName', StringType(), True),
    StructField('SheetName', StringType(), True),
    StructField('ModificationTime', StringType(), True),
    StructField('LoadTime', StringType(), True),
    StructField('Data', StringType(), True)
])

df = spark.createDataFrame(data, schema)  # Create a Spark DataFrame with explicit schema
display(df)

In [0]:
#create a table called delta_source
df.write.format("delta").mode("overwrite").saveAsTable("delta_source")

In [0]:
display(df)

In [0]:
df2 = df.filter((df['FileName'] == '/dbfs/user/arundhuti/delta/Dynamic_Excel_Source_Data/data_detail.xlsx') & (df['SheetName'] == 'Member')).select('Data')
display(df2)

In [0]:
import json

df3 = json.loads(df2.collect()[0]['Data'])
cols = df3['columns']
data = df3['data']
sdf = spark.createDataFrame(data, cols)
display(sdf)
#df3

In [0]:
current_time = time.strftime("%Y%m%d:%H:%M:%S",time.localtime())
current_time 


In [0]:
%sql
--select * from dev_edh.dummy.delta_source